# ⚡ Intermédiaire | ⏱ 45 min | 🔑 Concepts : sys.argv, argparse, typer

# Arguments de Ligne de Commande en Python

## 🎯 Objectifs

- Comprendre comment accéder aux arguments CLI avec `sys.argv`
- Maîtriser `argparse` pour créer des interfaces CLI robustes
- Découvrir `typer` comme alternative moderne
- Implémenter des sous-commandes et options avancées
- Connaître les pièges courants et bonnes pratiques

## 📋 Prérequis

- Python 3.8+
- Connaissances de base en Python (fonctions, types)
- Familiarité avec la ligne de commande

## 1. sys.argv : Accès basique aux arguments

`sys.argv` est la manière la plus simple d'accéder aux arguments de ligne de commande. C'est une liste contenant :
- `sys.argv[0]` : le nom du script
- `sys.argv[1:]` : les arguments passés

**Avantages** : Simple, pas de dépendance

**Inconvénients** : Parsing manuel, pas de validation, pas de help automatique

In [ ]:
import sys

# Simulation d'arguments (en Jupyter, sys.argv contient le chemin du kernel)
# Dans un vrai script, on aurait : python script.py arg1 arg2
sys.argv = ['script.py', 'fichier.txt', '--verbose']

print(f"Nom du script : {sys.argv[0]}")
print(f"Nombre d'arguments : {len(sys.argv) - 1}")
print(f"Arguments : {sys.argv[1:]}")

# Exemple d'usage basique
if len(sys.argv) < 2:
    print("Usage: python script.py <fichier>")
    sys.exit(1)

fichier = sys.argv[1]
verbose = '--verbose' in sys.argv

print(f"\nFichier à traiter : {fichier}")
print(f"Mode verbose : {verbose}")

## 2. argparse : Le module standard

`argparse` est le module standard pour créer des interfaces CLI professionnelles. Il offre :
- Parsing automatique des arguments
- Génération automatique du message d'aide
- Validation des types et valeurs
- Support des arguments positionnels et optionnels
- Sous-commandes (comme git : `git commit`, `git push`)

### 2.1 ArgumentParser basique

In [ ]:
import argparse

# Créer le parser
parser = argparse.ArgumentParser(
    prog='MonScript',
    description='Exemple de script avec argparse',
    epilog='Merci d\'utiliser ce script!'
)

# Argument positionnel (obligatoire)
parser.add_argument(
    'fichier',
    help='Chemin du fichier à traiter'
)

# Argument optionnel avec flag
parser.add_argument(
    '-v', '--verbose',
    action='store_true',
    help='Active le mode verbose'
)

# Argument optionnel avec valeur
parser.add_argument(
    '-o', '--output',
    default='output.txt',
    help='Fichier de sortie (défaut: output.txt)'
)

# Parser les arguments (simulation)
args = parser.parse_args(['data.csv', '--verbose', '-o', 'result.txt'])

print(f"Fichier d'entrée : {args.fichier}")
print(f"Fichier de sortie : {args.output}")
print(f"Mode verbose : {args.verbose}")

In [ ]:
# Afficher le message d'aide
parser.print_help()

### 2.2 Types, choix et validation

In [ ]:
parser_types = argparse.ArgumentParser(description='Validation avec argparse')

# Type int avec validation de plage
parser_types.add_argument(
    '--port',
    type=int,
    default=8000,
    help='Port du serveur (1-65535)'
)

# Choix limités
parser_types.add_argument(
    '--format',
    choices=['json', 'csv', 'xml'],
    default='json',
    help='Format de sortie'
)

# Type float
parser_types.add_argument(
    '--threshold',
    type=float,
    default=0.5,
    help='Seuil de confiance (0-1)'
)

# Liste de valeurs
parser_types.add_argument(
    '--tags',
    nargs='+',  # Au moins 1 valeur
    help='Tags à appliquer'
)

# Argument requis (optionnel mais obligatoire)
parser_types.add_argument(
    '--api-key',
    required=True,
    help='Clé API (obligatoire)'
)

# Parser avec validation
args = parser_types.parse_args([
    '--port', '3000',
    '--format', 'csv',
    '--threshold', '0.75',
    '--tags', 'python', 'data', 'engineering',
    '--api-key', 'secret123'
])

print(f"Port : {args.port} (type: {type(args.port).__name__})")
print(f"Format : {args.format}")
print(f"Threshold : {args.threshold} (type: {type(args.threshold).__name__})")
print(f"Tags : {args.tags}")
print(f"API Key : {args.api_key}")

### 2.3 Actions personnalisées

In [ ]:
parser_actions = argparse.ArgumentParser(description='Actions argparse')

# store_true / store_false
parser_actions.add_argument('--debug', action='store_true', help='Mode debug')
parser_actions.add_argument('--no-color', action='store_true', help='Désactive les couleurs')

# count : compte le nombre d'occurrences (-vvv = 3)
parser_actions.add_argument('-v', '--verbose', action='count', default=0, help='Niveau de verbosité')

# append : ajoute à une liste
parser_actions.add_argument('--exclude', action='append', help='Patterns à exclure')

# version
parser_actions.add_argument('--version', action='version', version='%(prog)s 1.0.0')

args = parser_actions.parse_args([
    '--debug',
    '-vvv',
    '--exclude', '*.pyc',
    '--exclude', '__pycache__'
])

print(f"Debug : {args.debug}")
print(f"Verbosité : {args.verbose}")
print(f"Exclusions : {args.exclude}")

### 2.4 Sous-commandes (subparsers)

Comme `git commit`, `git push`, `docker run`, etc.

In [ ]:
# Parser principal
parser_sub = argparse.ArgumentParser(prog='db-tool', description='Outil de gestion de base de données')
parser_sub.add_argument('--config', default='db.conf', help='Fichier de configuration')

# Sous-commandes
subparsers = parser_sub.add_subparsers(dest='command', help='Commandes disponibles')

# Commande 'migrate'
migrate_parser = subparsers.add_parser('migrate', help='Exécuter les migrations')
migrate_parser.add_argument('--target', default='latest', help='Version cible')
migrate_parser.add_argument('--dry-run', action='store_true', help='Simulation')

# Commande 'backup'
backup_parser = subparsers.add_parser('backup', help='Sauvegarder la base')
backup_parser.add_argument('output', help='Fichier de sauvegarde')
backup_parser.add_argument('--compress', action='store_true', help='Compresser')

# Commande 'restore'
restore_parser = subparsers.add_parser('restore', help='Restaurer depuis une sauvegarde')
restore_parser.add_argument('backup_file', help='Fichier de sauvegarde')
restore_parser.add_argument('--force', action='store_true', help='Forcer la restauration')

# Test avec la commande 'migrate'
args = parser_sub.parse_args(['--config', 'prod.conf', 'migrate', '--target', 'v2.0', '--dry-run'])
print(f"Commande : {args.command}")
print(f"Config : {args.config}")
print(f"Target : {args.target}")
print(f"Dry run : {args.dry_run}")

print("\n" + "="*50 + "\n")

# Test avec la commande 'backup'
args = parser_sub.parse_args(['backup', 'db_backup.sql', '--compress'])
print(f"Commande : {args.command}")
print(f"Output : {args.output}")
print(f"Compress : {args.compress}")

## 3. Typer : Alternative moderne

`typer` est une bibliothèque moderne qui utilise les type hints Python pour créer des CLI. Elle est construite sur `click`.

**Avantages** :
- Code très concis
- Utilise les type hints (validation automatique)
- Auto-complétion shell
- Documentation automatique
- Moins de boilerplate

**Installation** : `pip install typer`

In [ ]:
# Note: typer nécessite une installation pip
# pip install typer

try:
    import typer
    from typing import Optional, List
    from enum import Enum

    # Définir un enum pour les choix
    class Format(str, Enum):
        json = "json"
        csv = "csv"
        xml = "xml"

    def process_file(
        fichier: str,
        output: str = "output.txt",
        verbose: bool = False,
        format: Format = Format.json,
        tags: Optional[List[str]] = None
    ):
        """
        Traite un fichier avec différentes options.
        
        Args:
            fichier: Chemin du fichier à traiter
            output: Fichier de sortie
            verbose: Active le mode verbose
            format: Format de sortie (json, csv, xml)
            tags: Liste de tags
        """
        print(f"📁 Fichier : {fichier}")
        print(f"💾 Output : {output}")
        print(f"📢 Verbose : {verbose}")
        print(f"📋 Format : {format.value}")
        print(f"🏷️  Tags : {tags or []}")

    # Simulation de l'appel (normalement fait avec typer.run())
    print("Avec Typer, le code est beaucoup plus simple !")
    print("\nExemple d'appel :")
    process_file(
        fichier="data.csv",
        output="result.txt",
        verbose=True,
        format=Format.csv,
        tags=["python", "data"]
    )
    
except ImportError:
    print("⚠️  Typer n'est pas installé.")
    print("Installez-le avec : pip install typer")
    print("\nExemple de code Typer :")
    print("""
import typer
from typing import Optional

def main(
    fichier: str,
    verbose: bool = False,
    output: str = "output.txt"
):
    \"\"\"Traite un fichier.\"\"\"  
    print(f"Traitement de {fichier}...")
    if verbose:
        print("Mode verbose activé")

if __name__ == "__main__":
    typer.run(main)
    """)

### 3.1 Typer avec sous-commandes

In [ ]:
# Exemple de structure avec sous-commandes en Typer
code_example = '''
import typer

app = typer.Typer()

@app.command()
def migrate(
    target: str = "latest",
    dry_run: bool = False
):
    """Exécuter les migrations de base de données."""
    typer.echo(f"Migration vers {target}...")
    if dry_run:
        typer.echo("Mode simulation (dry-run)")

@app.command()
def backup(
    output: str,
    compress: bool = False
):
    """Sauvegarder la base de données."""
    typer.echo(f"Sauvegarde vers {output}...")
    if compress:
        typer.echo("Compression activée")

@app.command()
def restore(
    backup_file: str,
    force: bool = False
):
    """Restaurer depuis une sauvegarde."""
    if not force:
        if not typer.confirm("Êtes-vous sûr ?"):
            typer.echo("Annulé.")
            raise typer.Abort()
    typer.echo(f"Restauration depuis {backup_file}...")

if __name__ == "__main__":
    app()
'''

print("Structure Typer avec sous-commandes :")
print(code_example)

print("\nUtilisation :")
print("  python db_tool.py migrate --target v2.0 --dry-run")
print("  python db_tool.py backup output.sql --compress")
print("  python db_tool.py restore backup.sql --force")

## 4. Comparaison : argparse vs typer vs click

| Critère | argparse | typer | click |
|---------|----------|-------|-------|
| **Installation** | Standard library | `pip install typer` | `pip install click` |
| **Verbosité** | Moyen | Faible | Moyen |
| **Type hints** | Non | Oui ✅ | Non |
| **Sous-commandes** | Oui (complexe) | Oui (simple) | Oui (simple) |
| **Auto-complétion** | Non | Oui ✅ | Oui |
| **Testing** | Difficile | Facile | Facile |
| **Documentation** | Manuel | Auto | Manuel |
| **Courbe d'apprentissage** | Moyenne | Faible | Moyenne |

**Recommandations** :
- **argparse** : Scripts simples, pas de dépendance externe
- **typer** : Applications CLI modernes, équipes Python 3.6+
- **click** : Si vous avez déjà des outils click, ou préférez les décorateurs

## 5. Pattern : Script CLI bien structuré

In [ ]:
import argparse
import sys
import logging

def setup_logging(verbose: bool) -> None:
    """Configure le logging selon le niveau de verbosité."""
    level = logging.DEBUG if verbose else logging.INFO
    logging.basicConfig(
        level=level,
        format='%(asctime)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )

def process_data(input_file: str, output_file: str, format: str) -> None:
    """Logique métier du script."""
    logging.info(f"Traitement de {input_file}...")
    logging.debug(f"Format de sortie : {format}")
    # ... traitement ...
    logging.info(f"Résultat écrit dans {output_file}")

def parse_args(argv=None) -> argparse.Namespace:
    """Parse les arguments de ligne de commande."""
    parser = argparse.ArgumentParser(
        description='Outil de traitement de données',
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    
    parser.add_argument('input', help='Fichier d\'entrée')
    parser.add_argument('-o', '--output', default='output.txt', help='Fichier de sortie')
    parser.add_argument('-f', '--format', choices=['json', 'csv'], default='json')
    parser.add_argument('-v', '--verbose', action='store_true', help='Mode verbose')
    parser.add_argument('--version', action='version', version='1.0.0')
    
    return parser.parse_args(argv)

def main(argv=None) -> int:
    """Point d'entrée principal."""
    try:
        args = parse_args(argv)
        setup_logging(args.verbose)
        
        logging.info("Démarrage du traitement...")
        process_data(args.input, args.output, args.format)
        logging.info("Traitement terminé avec succès")
        
        return 0  # Exit code : succès
        
    except FileNotFoundError as e:
        logging.error(f"Fichier introuvable : {e}")
        return 1
    except Exception as e:
        logging.exception(f"Erreur inattendue : {e}")
        return 2

if __name__ == '__main__':
    sys.exit(main())

# Test en Jupyter
exit_code = main(['input.txt', '-o', 'output.json', '-f', 'json', '--verbose'])
print(f"\nExit code : {exit_code}")

## 🚨 Pièges courants

### 1. Oublier parse_args()

In [ ]:
# ❌ MAUVAIS : oublie d'appeler parse_args()
parser = argparse.ArgumentParser()
parser.add_argument('--name')
# parser contient la configuration, pas les valeurs parsées !
# print(parser.name)  # AttributeError

# ✅ BON : appeler parse_args()
parser = argparse.ArgumentParser()
parser.add_argument('--name', default='World')
args = parser.parse_args(['--name', 'Python'])
print(f"Hello, {args.name}!")  # Fonctionne

### 2. Confusion positional vs optional

In [ ]:
parser = argparse.ArgumentParser()

# Positionnel : PAS de tiret, OBLIGATOIRE (sauf si nargs='?')
parser.add_argument('fichier', help='Fichier à traiter')

# Optionnel : commence par - ou --, optionnel par défaut
parser.add_argument('--output', help='Fichier de sortie')
parser.add_argument('-v', '--verbose', action='store_true')

# ❌ ERREUR : argument positionnel manquant
try:
    args = parser.parse_args(['--output', 'out.txt'])
except SystemExit:
    print("❌ Erreur : l'argument positionnel 'fichier' est obligatoire")

# ✅ BON : fournir l'argument positionnel
args = parser.parse_args(['input.txt', '--output', 'out.txt'])
print(f"✅ Fichier : {args.fichier}, Output : {args.output}")

### 3. Message d'aide incomplet

In [ ]:
# ❌ MAUVAIS : pas de help
parser_bad = argparse.ArgumentParser()
parser_bad.add_argument('--threshold', type=float)
parser_bad.add_argument('--format')

# ✅ BON : avec help et valeurs par défaut explicites
parser_good = argparse.ArgumentParser(
    description='Script de traitement avec documentation',
    formatter_class=argparse.ArgumentDefaultsHelpFormatter  # Affiche les valeurs par défaut
)
parser_good.add_argument(
    '--threshold',
    type=float,
    default=0.5,
    help='Seuil de confiance entre 0 et 1'
)
parser_good.add_argument(
    '--format',
    choices=['json', 'csv'],
    default='json',
    help='Format de sortie'
)

print("Comparaison des messages d'aide :\n")
print("="*60)
print("MAUVAIS (sans help) :")
parser_bad.print_help()
print("\n" + "="*60)
print("BON (avec help détaillé) :")
parser_good.print_help()

### 4. Gestion des erreurs

In [ ]:
# ❌ MAUVAIS : laisse argparse appeler sys.exit()
def bad_main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--required', required=True)
    args = parser.parse_args()  # Appelle sys.exit(2) si erreur
    # Impossible de gérer l'erreur !

# ✅ BON : capturer les erreurs pour un contrôle fin
def good_main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--required', required=True)
    
    try:
        args = parser.parse_args(['--required', 'value'])
        print(f"✅ Valeur reçue : {args.required}")
        return 0
    except SystemExit as e:
        if e.code != 0:
            print(f"❌ Erreur de parsing : exit code {e.code}")
        return e.code

exit_code = good_main()
print(f"Exit code : {exit_code}")

## 💪 Mini-exercices

### Exercice 1 : Calculatrice CLI avec argparse

Créer un script de calculatrice en ligne de commande qui :
- Accepte deux nombres (arguments positionnels)
- Accepte une opération (`--operation` : add, sub, mul, div)
- A un flag `--round` pour arrondir le résultat
- Affiche le résultat

In [ ]:
# À vous de jouer !
import argparse

def calculer(a: float, b: float, operation: str) -> float:
    """Effectue l'opération demandée."""
    # TODO : implémenter la logique
    pass

def main_calculatrice(argv=None):
    parser = argparse.ArgumentParser(description='Calculatrice CLI')
    
    # TODO : ajouter les arguments
    
    args = parser.parse_args(argv)
    
    # TODO : calculer et afficher le résultat
    pass

# Test
# main_calculatrice(['10', '3', '--operation', 'div', '--round'])

### Solution Exercice 1

In [ ]:
import argparse
from typing import Optional

def calculer(a: float, b: float, operation: str) -> float:
    """Effectue l'opération demandée."""
    operations = {
        'add': lambda x, y: x + y,
        'sub': lambda x, y: x - y,
        'mul': lambda x, y: x * y,
        'div': lambda x, y: x / y if y != 0 else float('inf')
    }
    return operations[operation](a, b)

def main_calculatrice(argv=None):
    parser = argparse.ArgumentParser(
        description='Calculatrice en ligne de commande',
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    
    # Arguments positionnels
    parser.add_argument('a', type=float, help='Premier nombre')
    parser.add_argument('b', type=float, help='Deuxième nombre')
    
    # Arguments optionnels
    parser.add_argument(
        '-o', '--operation',
        choices=['add', 'sub', 'mul', 'div'],
        default='add',
        help='Opération à effectuer'
    )
    parser.add_argument(
        '--round',
        action='store_true',
        help='Arrondir le résultat à l\'entier le plus proche'
    )
    
    args = parser.parse_args(argv)
    
    # Calcul
    resultat = calculer(args.a, args.b, args.operation)
    
    # Arrondir si demandé
    if args.round:
        resultat = round(resultat)
    
    # Affichage
    symboles = {'add': '+', 'sub': '-', 'mul': '*', 'div': '/'}
    print(f"{args.a} {symboles[args.operation]} {args.b} = {resultat}")
    
    return resultat

# Tests
print("Test 1 : Addition")
main_calculatrice(['10', '5', '--operation', 'add'])

print("\nTest 2 : Division avec arrondi")
main_calculatrice(['10', '3', '--operation', 'div', '--round'])

print("\nTest 3 : Multiplication")
main_calculatrice(['7.5', '4', '--operation', 'mul'])

### Exercice 2 : Refactoring en Typer

Réécrire la calculatrice de l'exercice 1 en utilisant Typer.

In [ ]:
# Code équivalent en Typer (pour référence)
code_typer = '''
import typer
from enum import Enum

class Operation(str, Enum):
    add = "add"
    sub = "sub"
    mul = "mul"
    div = "div"

def calculer(a: float, b: float, operation: Operation) -> float:
    operations = {
        Operation.add: lambda x, y: x + y,
        Operation.sub: lambda x, y: x - y,
        Operation.mul: lambda x, y: x * y,
        Operation.div: lambda x, y: x / y if y != 0 else float(\'inf\')
    }
    return operations[operation](a, b)

def main(
    a: float,
    b: float,
    operation: Operation = Operation.add,
    round_result: bool = typer.Option(False, "--round", help="Arrondir le résultat")
):
    """Calculatrice en ligne de commande."""
    resultat = calculer(a, b, operation)
    
    if round_result:
        resultat = round(resultat)
    
    symboles = {Operation.add: \'+\', Operation.sub: \'-\', Operation.mul: \'*\', Operation.div: \'/\'}
    typer.echo(f"{a} {symboles[operation]} {b} = {resultat}")

if __name__ == "__main__":
    typer.run(main)
'''

print("Version Typer de la calculatrice :")
print(code_typer)

print("\n" + "="*60)
print("Comparaison :")
print("  argparse : ~50 lignes de code")
print("  typer    : ~30 lignes de code")
print("\nAvantages Typer :")
print("  - Type hints natifs")
print("  - Moins de boilerplate")
print("  - Validation automatique des types")
print("  - Documentation générée depuis les docstrings")

### Exercice 3 : Script avec sous-commandes

Créer un outil `file-tool` avec 3 sous-commandes :
1. `count <fichier>` : compte les lignes, mots, caractères
2. `search <fichier> <pattern>` : recherche un pattern (simulation)
3. `convert <fichier> --to-upper/--to-lower` : convertit en majuscules/minuscules

In [ ]:
# À vous de jouer !
import argparse

def cmd_count(fichier: str) -> None:
    """Compte lignes, mots, caractères."""
    # TODO
    pass

def cmd_search(fichier: str, pattern: str) -> None:
    """Recherche un pattern."""
    # TODO
    pass

def cmd_convert(fichier: str, to_upper: bool, to_lower: bool) -> None:
    """Convertit le texte."""
    # TODO
    pass

def main_filetool(argv=None):
    # TODO : créer le parser avec subparsers
    pass

# Test
# main_filetool(['count', 'test.txt'])

### Solution Exercice 3

In [ ]:
import argparse

def cmd_count(fichier: str) -> None:
    """Compte lignes, mots, caractères."""
    # Simulation avec du texte fictif
    texte = "Ceci est un exemple.\nUne deuxième ligne.\nEt une troisième."
    
    lignes = texte.count('\n') + 1
    mots = len(texte.split())
    caracteres = len(texte)
    
    print(f"📄 Fichier : {fichier}")
    print(f"   Lignes     : {lignes}")
    print(f"   Mots       : {mots}")
    print(f"   Caractères : {caracteres}")

def cmd_search(fichier: str, pattern: str, case_insensitive: bool = False) -> None:
    """Recherche un pattern."""
    texte = "Python est un langage formidable.\nJ'adore Python !\nPython rocks."
    lignes = texte.split('\n')
    
    print(f"🔍 Recherche de '{pattern}' dans {fichier}")
    if case_insensitive:
        print("   (insensible à la casse)")
    
    matches = 0
    for i, ligne in enumerate(lignes, 1):
        texte_recherche = ligne.lower() if case_insensitive else ligne
        pattern_recherche = pattern.lower() if case_insensitive else pattern
        
        if pattern_recherche in texte_recherche:
            print(f"   Ligne {i}: {ligne}")
            matches += 1
    
    print(f"\n✅ {matches} correspondance(s) trouvée(s)")

def cmd_convert(fichier: str, to_upper: bool, to_lower: bool) -> None:
    """Convertit le texte."""
    texte = "Ceci Est Un Exemple De Texte Mixte."
    
    if to_upper and to_lower:
        print("❌ Erreur : --to-upper et --to-lower sont mutuellement exclusifs")
        return
    
    print(f"📝 Conversion de {fichier}")
    print(f"   Original : {texte}")
    
    if to_upper:
        resultat = texte.upper()
        print(f"   Résultat : {resultat}")
    elif to_lower:
        resultat = texte.lower()
        print(f"   Résultat : {resultat}")
    else:
        print("   Aucune conversion spécifiée")

def main_filetool(argv=None):
    """Point d'entrée principal."""
    parser = argparse.ArgumentParser(
        prog='file-tool',
        description='Outil de manipulation de fichiers texte'
    )
    
    subparsers = parser.add_subparsers(dest='command', help='Commandes disponibles')
    
    # Commande 'count'
    count_parser = subparsers.add_parser('count', help='Compter lignes/mots/caractères')
    count_parser.add_argument('fichier', help='Fichier à analyser')
    
    # Commande 'search'
    search_parser = subparsers.add_parser('search', help='Rechercher un pattern')
    search_parser.add_argument('fichier', help='Fichier où chercher')
    search_parser.add_argument('pattern', help='Pattern à rechercher')
    search_parser.add_argument(
        '-i', '--ignore-case',
        action='store_true',
        help='Insensible à la casse'
    )
    
    # Commande 'convert'
    convert_parser = subparsers.add_parser('convert', help='Convertir le texte')
    convert_parser.add_argument('fichier', help='Fichier à convertir')
    convert_parser.add_argument('--to-upper', action='store_true', help='En majuscules')
    convert_parser.add_argument('--to-lower', action='store_true', help='En minuscules')
    
    args = parser.parse_args(argv)
    
    # Dispatch vers la bonne commande
    if args.command == 'count':
        cmd_count(args.fichier)
    elif args.command == 'search':
        cmd_search(args.fichier, args.pattern, args.ignore_case)
    elif args.command == 'convert':
        cmd_convert(args.fichier, args.to_upper, args.to_lower)
    else:
        parser.print_help()

# Tests
print("Test 1 : count")
main_filetool(['count', 'document.txt'])

print("\n" + "="*60 + "\n")
print("Test 2 : search")
main_filetool(['search', 'code.py', 'Python', '-i'])

print("\n" + "="*60 + "\n")
print("Test 3 : convert")
main_filetool(['convert', 'text.txt', '--to-upper'])

## 📚 Ressources complémentaires

- [Documentation argparse](https://docs.python.org/3/library/argparse.html)
- [Documentation Typer](https://typer.tiangolo.com/)
- [Documentation Click](https://click.palletsprojects.com/)
- [Real Python - argparse Tutorial](https://realpython.com/command-line-interfaces-python-argparse/)
- [Python CLI Best Practices](https://clig.dev/)